In [1]:
!pip install -U ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO
import os, shutil, pathlib, glob, zipfile
import yaml
import cv2
from IPython.display import Image, display, Video


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
model_path = "/content/drive/MyDrive/Train model/best.pt"
model = YOLO(model_path)


In [5]:
def predict_video(video_path, out_dir="/content/video_results", conf=0.25):
    # Ensure the output directory exists
    os.makedirs(out_dir, exist_ok=True)
    print(f"Running prediction on {video_path}, saving to {out_dir}")
    model.track(
        source=video_path,
        persist=True,
        tracker="bytetrack.yaml",
        conf=conf,
        save=True,
        save_dir=out_dir, # This should instruct Ultralytics to save to this directory
        project=out_dir, # Explicitly set project to out_dir to ensure it saves within it
        name="tracked_video" # Give a specific name to the output run
    )
    # After model.track completes, it should print the location where the results were saved.
    # We will search for the video based on the expected Ultralytics output structure.
    return out_dir

video_path = "/content/drive/MyDrive/cctv.mp4/market.mp4"
out_dir = predict_video(video_path)

# Try to find the video files in the specified output directory.

expected_save_base = os.path.join(out_dir, "tracked_video")
print(f"Checking for video files in: {expected_save_base}")
video_files = glob.glob(f"{expected_save_base}/**/*.mp4", recursive=True)

if video_files:
    print(f"Found video files: {video_files}")
    display(Video(video_files[0], embed=True))
else:
    # If no video found in the specific expected path, broaden the search to the general output directory
    print(f"No video files found in {expected_save_base}. Searching all .mp4 files under {out_dir}.")
    video_files_broad_search = glob.glob(f"{out_dir}/**/*.mp4", recursive=True)
    if video_files_broad_search:
        print(f"Found video files during broad search: {video_files_broad_search}")
        display(Video(video_files_broad_search[0], embed=True))
    else:
        # As a last resort, check the default Ultralytics runs directory structure if 'save_dir' was completely ignored
        print(f"No .mp4 video file found under {out_dir}. Checking default Ultralytics runs directory.")
        fallback_video_files = glob.glob("/content/runs/track/**/*.mp4", recursive=True)
        if fallback_video_files:
            print(f"Found video files in fallback /content/runs/track location: {fallback_video_files}")
            display(Video(fallback_video_files[0], embed=True))
        else:
            print("No .mp4 video found anywhere in expected or default Ultralytics output locations.")

Running prediction on /content/drive/MyDrive/cctv.mp4/market.mp4, saving to /content/video_results
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 228ms
Prepared 1 package in 81ms
Installed 1 package in 1ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.8s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

vi